# Binary CSP solver

*Guillaume DALLE*

## 1. CSP structure and backtracking

### 1.1) CSP object

In [1]:
mutable struct CSP
    n_variables::Int64
    names::Array{String, 1}
    domain_sizes::Array{Int64, 1}
    
    possible_value::Array{Array{Bool, 1}, 1}
    value_scores::Array{Array{Int64, 1}, 1}
    value_order::Array{Array{Int64, 1}}
        
    n_constraints::Int64
    constraint_var1::Array{Int64, 1}
    constraint_var2::Array{Int64, 1}
    constraint_satisfaction::Array{Array{Bool, 2}, 1}
    
    var1_constraints::Array{Array{Int64, 1}, 1}
    var2_constraints::Array{Array{Int64, 1}, 1}
    
    CSP() = new(
        0,
        String[],
        Int64[],
        
        Array{Bool, 1}[],
        Array{Int64, 1}[],
        Array{Int64, 1}[],
        
        0,
        Int64[],
        Int64[],
        Array{Bool, 1}[],
        
        Array{Int64, 1}[],
        Array{Int64, 1}[],
    )
end

### 1.2) Variables and constraints

In [2]:
function add_variable!(csp::CSP, name::String, domain_size::Int64)
    csp.n_variables += 1
    push!(csp.names, name)
    push!(csp.domain_sizes, domain_size)
    push!(csp.possible_value, ones(Bool, domain_size))
    push!(csp.value_scores, zeros(Int64, domain_size))
    push!(csp.value_order, collect(1:domain_size))
    
    push!(csp.var1_constraints, Int64[])
    push!(csp.var2_constraints, Int64[])
end

add_variable! (generic function with 1 method)

In [3]:
function add_constraint!(csp::CSP, var1::Int64, domain::Array{Int64, 1})
    for value1 in 1:csp.domain_sizes[var1]
        if !(value1 in domain)
            csp.possible_value[var1][value1] = false
        end
    end    
end

add_constraint! (generic function with 1 method)

In [4]:
function add_constraint!(csp::CSP, var1::Int64, var2::Int64, compatibility::Function)
    csp.n_constraints += 1

    m1::Int64 = csp.domain_sizes[var1]
    m2::Int64 = csp.domain_sizes[var2]
    satisfaction::Array{Bool, 2} = zeros(Bool, m1, m2)
    for value1 in 1:m1, value2 in 1:m2
        if compatibility(csp, var1, var2, value1, value2)
            satisfaction[value1, value2] = true
            csp.value_scores[var1][value1] += 1
            csp.value_scores[var2][value2] += 1
        end
    end
    
    push!(csp.constraint_var1, var1)
    push!(csp.constraint_var2, var2)
    push!(csp.constraint_satisfaction, satisfaction)
    
    push!(csp.var1_constraints[var1], csp.n_constraints)
    push!(csp.var2_constraints[var2], csp.n_constraints)
    
end

add_constraint! (generic function with 2 methods)

### 1.3) Consistency check

In [5]:
function check_feasibility(csp::CSP, instantiation::Array{Int64, 1}, new_var::Int64)
    if new_var == -1
        return true
    end
    for cons in csp.var1_constraints[new_var]
        var2::Int64 = csp.constraint_var2[cons]
        satisfaction = csp.constraint_satisfaction[cons]
        value1 = instantiation[new_var]
        value2 = instantiation[var2]
        if (value1 !== -1) && (value2 !== -1)
            if !satisfaction[value1, value2]
                return false
            end
        end
    end
    return true
end

check_feasibility (generic function with 1 method)

### 1.4) Choice of variable / value

In [6]:
function evaluate_variable(csp::CSP, var::Int64, criterion::String)
    if criterion == "domain_size"
        current_domain_size::Int64 = 0
        for value in 1:csp.domain_sizes[var]
            if csp.possible_value[var][value]
                current_domain_size += 1
            end
        end
        return - current_domain_size + rand()
    elseif criterion == "constraints"
        return length(csp.var1_constraints[var]) + rand()
    end
end

function choose_next_variable(csp::CSP, instantiation::Array{Int64, 1}, criterion="domain_size")
    best_var::Int64 = -1
    best_eval::Float64 = -Inf
        
    for var in 1:csp.n_variables
        if instantiation[var] != -1
            continue
        else
            eval = evaluate_variable(csp, var, criterion)
            if eval > best_eval
                best_var = var
                best_eval = eval
            end
        end
    end
    return best_var
end

choose_next_variable (generic function with 2 methods)

In [7]:
function sort_values(csp::CSP)
    for var in 1:csp.n_variables
        value_sorting_criterion(value::Int64) = csp.value_scores[var][value]
        csp.value_order[var] = sort(csp.value_order[var], by=value_sorting_criterion)
    end
end

sort_values (generic function with 1 method)

### 1.4) Look-ahead

In [8]:
function do_look_ahead!(
        csp::CSP, instantiation::Array{Int64, 1}, new_var::Int64,
        look_ahead_method::String)
    if look_ahead_method == "FC"
        to_discard = forward_checking(csp, instantiation, new_var)
    elseif look_ahead_method == "MAC3"
        to_discard = AC3(csp, instantiation, new_var)
    else
        to_discard = Tuple{Int64, Int64}[]
    end
    for (var, value) in to_discard
        csp.possible_value[var][value] = false
    end
    return to_discard
end

function undo_look_ahead!(csp::CSP, discarded::Array{Tuple{Int64, Int64}, 1})
    for (var, value) in discarded
        csp.possible_value[var][value] = true
    end
end

undo_look_ahead! (generic function with 1 method)

In [9]:
function forward_checking(csp::CSP, instantiation::Array{Int64, 1}, x::Int64)
    to_discard::Array{Tuple{Int64, Int64}, 1} = Tuple{Int64, Int64}[]
    
    a::Int64 = instantiation[x]
    for aa in 1:csp.domain_sizes[x]
        if (aa != a) && csp.possible_value[x][aa]
            push!(to_discard, (x, aa))
        end
    end
    
    for cons in csp.var1_constraints[x]
        y = csp.constraint_var2[cons]
        if instantiation[y] != -1
            continue
        end
        
        for b in 1:csp.domain_sizes[y]
            if (!csp.constraint_satisfaction[cons][a, b]) && csp.possible_value[y][b]
                push!(to_discard, (y, b))
            end
        end
    end
    
    return to_discard
end

forward_checking (generic function with 1 method)

In [10]:
function AC3(csp::CSP, instantiation::Array{Int64, 1}, new_var::Int64)
    to_discard::Array{Tuple{Int64, Int64}, 1} = Tuple{Int64, Int64}[]
    
    for other_value in 1:csp.domain_sizes[new_var]
        if (other_value != instantiation[new_var]) && csp.possible_value[new_var][other_value]
            push!(to_discard, (new_var, other_value))
        end
    end
    
    to_test = Set{Int64}()
    for cons in 1:csp.n_constraints
        push!(to_test, cons)
    end
    
    while !isempty(to_test)
        cons = pop!(to_test)
        x = csp.constraint_var1[cons]
        y = csp.constraint_var2[cons]
        for a in 1:csp.domain_sizes[x]
            
            if !csp.possible_value[x][a]
                continue
            end
            
            supported::Bool = false
            for b in 1:csp.domain_sizes[y]
                if !csp.possible_value[y][b]
                    continue
                end
                if csp.constraint_satisfaction[cons][a, b]
                    supported = true
                    break
                end
            end
            
            if !supported
                push!(to_discard, (x, a))
                csp.possible_value[x][a] = false
                for cons_impacted in csp.var2_constraints[x]
                    if csp.constraint_var1[cons] != y
                        push!(to_test, cons_impacted)
                    end
                end
            end
        end
    end
    
    return to_discard
end

AC3 (generic function with 1 method)

In [11]:
function initAC4(csp::CSP)
    Q::Set{Tuple{Int64, Int64}} = Set()
    S::Dict{Tuple{Int64, Int64}, Set{Tuple{Int64, Int64}}} = Dict()   
    count::Dict{Tuple{Int64, Int64, Int64}, Int64} = Dict()
    
    for cons in 1:csp.n_constraints
        x::Int64 = csp.constraint_var1[cons]
        y::Int64 = csp.constraint_var2[cons]
        total::Int64 = 0
        
        for a in 1:csp.domain_sizes[x]
            if !csp.possible_value[x][a]
                continue
            end
            
            for b in 1:csp.domain_sizes[y]
                if !csp.possible_value[y][b]
                    continue
                end
                
                if csp.constraint_satisfaction[cons][a, b]
                    total += 1
                    if (y, b) in keys(S)
                        push!(S[(y, b)], (x, a))
                    else
                        S[(y, b)] = Set([(x, a)])
                    end
                end
            end
            count[(x, y, a)] = total
            if count[(x, y, a)] == 0
                csp.possible_value[x][a] = false
                push!(Q, (x, a))
            end
        end
    end
    return Q, S, count
end

function AC4(csp::CSP)
    return 0
end

AC4 (generic function with 1 method)

### 1.5) Backtracking

In [12]:
function backtrack!(
        csp::CSP,
        instantiation::Array{Int64, 1},
        new_var::Int64,
        nodes_explored::Int64,
        look_ahead_method::String
    )
    if !check_feasibility(csp, instantiation, new_var)
        return (false, instantiation, nodes_explored)
    end
    
    new_var = choose_next_variable(csp, instantiation)
    if new_var == -1
        return true, instantiation, nodes_explored
    end
        
    for new_value in csp.value_order[new_var]
                
        if !csp.possible_value[new_var][new_value]
            continue
        end

        nodes_explored += 1
        instantiation[new_var] = new_value
        discarded = do_look_ahead!(csp, instantiation, new_var, look_ahead_method)
                
        (solution_found, solution, nodes_explored) = backtrack!(
            csp, instantiation, new_var, nodes_explored, look_ahead_method)
        if solution_found
            return (true, solution, nodes_explored)
        end
        
        instantiation[new_var] = -1
        undo_look_ahead!(csp, discarded)

    end
    
    return false, instantiation, nodes_explored
end

backtrack! (generic function with 1 method)

In [13]:
function backtrack!(csp::CSP, look_ahead_method::String)
    return backtrack!(
        csp, -ones(Int64, csp.n_variables), -1, 0, look_ahead_method)
end

function backtrack!(csp::CSP)
    return backtrack!(
        csp, -ones(Int64, csp.n_variables), -1, 0, "FC")
end

backtrack! (generic function with 3 methods)

In [14]:
function solve(csp::CSP)
    (status, instantiation, nodes_explored) = backtrack!(csp)
    if !status
        println("\nStatus: unfeasible")
    else
        println("\nStatus: solved")
        println("\nNodes explored: ", nodes_explored)
        println("\nSolution: ")
        for var in 1:csp.n_variables
            println(csp.names[var], ": ", instantiation[var])
        end
    end
    println()
    instantiation_dict::Dict{String, Int64} = Dict(
        csp.names[var] => instantiation[var] for var in 1:csp.n_variables)
    return instantiation_dict
end

solve (generic function with 1 method)

## 2. Applications

### 2.1) N-queens problem

In [15]:
function compatibility_nqueens(csp::CSP, var1::Int64, var2::Int64, value1::Int64, value2::Int64)
    # Exclude same row, same diagonal, same antidiagonal
    if (value1 == value2) || abs(value1 - value2) == abs(var1 - var2)
        return false
    else
        return true
    end
end

function symmetry_redundent_nqueens(csp::CSP, var1::Int64, var2::Int64, value1::Int64, value2::Int64)
    # Horizontal symmetry
    if (var1 == 1) && (var2 == 2)
        if value1 < value2
            return true
        end
    end
    return false
end

function define_nqueens(n::Int64)
    csp = CSP()
    for i in 1:n
        name = "row_" * string(i)
        domain_size = n
        add_variable!(csp, name, domain_size)
    end
    for i in 1:n, j in 1:n
        if (i != j)
            add_constraint!(csp, i, j, compatibility_nqueens)
        end
    end
    return csp
end

function visualize_nqueens(instantiation_dict::Dict{String, Int64})
    n::Int64 = length(instantiation_dict)
    for j in 1:n
        println()
        for i in 1:n
            if instantiation_dict["row_" * string(j)] == i
                print(" Q")
            else
                print(" .")
            end
        end
    end
end   

visualize_nqueens (generic function with 1 method)

In [16]:
@time big_nqueens = define_nqueens(100);

  0.589915 seconds (226.31 k allocations: 108.185 MiB, 13.59% gc time)


In [17]:
@time b, i, n = backtrack!(big_nqueens)

  0.241354 seconds (411.95 k allocations: 20.600 MiB)


(true, [62, 16, 9, 54, 15, 47, 64, 67, 20, 17  …  45, 40, 32, 3, 31, 84, 72, 94, 98, 91], 113)

In [18]:
nqueens_small = define_nqueens(10);
instantiation_dict = solve(nqueens_small)
visualize_nqueens(instantiation_dict)


Status: solved

Nodes explored: 19

Solution: 
row_1: 2
row_2: 5
row_3: 8
row_4: 4
row_5: 7
row_6: 10
row_7: 3
row_8: 1
row_9: 6
row_10: 9


 . Q . . . . . . . .
 . . . . Q . . . . .
 . . . . . . . Q . .
 . . . Q . . . . . .
 . . . . . . Q . . .
 . . . . . . . . . Q
 . . Q . . . . . . .
 Q . . . . . . . . .
 . . . . . Q . . . .
 . . . . . . . . Q .

### 2.2) Knight's tour

In [19]:
function coord_from_square(square::Int64, board_size::Int64)
    row = div(square - 1, board_size) + 1
    col = rem(square - 1, board_size) + 1
    return (row, col)
end

function square_from_coord(row::Int64, col::Int64, board_size::Int64)
    square = (row-1) * board_size + col
    return square
end

square_from_coord (generic function with 1 method)

In [20]:
function rook_move(square1::Int64, square2::Int64, n::Int64)
    row1, col1 = coord_from_square(square1, n)
    row2, col2 = coord_from_square(square2, n)
    return (square1 != square2) && ((row1 == row2) || (col1 == col2))
end

function bishop_move(square1::Int64, square2::Int64, n::Int64)
    row1, col1 = coord_from_square(square1, n)
    row2, col2 = coord_from_square(square2, n)
    return (square1 != square2) && (abs(row1 - row2) == abs(col1 - col2))
end

function knight_move(square1::Int64, square2::Int64, n::Int64)
    row1, col1 = coord_from_square(square1, n)
    row2, col2 = coord_from_square(square2, n)
    return (
        (abs(row1 - row2) == 1 && abs(col1 - col2) == 2) ||
        (abs(row1 - row2) == 2 && abs(col1 - col2) == 1)
    )
end

function queen_move(square1::Int64, square2::Int64, n::Int64)
    return rook_move(square1, square2, n) || bishop_move(square1, square2, n)
end

function king_move(square1::Int64, square2::Int64, n::Int64)
    row1, col1 = coord_from_square(square1, n)
    row2, col2 = coord_from_square(square2, n)
    return (square1 != square2) && (max(abs(row1 - row2), abs(col1 - col2)) == 1)
end

function piece_move(square1::Int64, square2::Int64, n::Int64, piece::String)
    if piece == "knight"
        return knight_move(square1, square2, n)
    elseif piece == "bishop"
        return bishop_move(square1, square2, n)
    elseif piece == "rook"
        return rook_move(square1, square2, n)
    elseif piece == "queen"
        return queen_move(square1, square2, n)
    elseif piece == "king"
        return king_move(square1, square2, n)
    end
end

piece_move (generic function with 1 method)

In [21]:
function compatibility_tour(
        csp::CSP,
        var1::Int64, var2::Int64,
        value1::Int64, value2::Int64,
        n::Int64, piece::String
    )
    if abs(var1 - var2) > 1
        return value1 != value2
    else
        return piece_move(value1, value2, n, piece)
    end
end

function define_tour(n::Int64, piece::String)
    csp = CSP()
    if piece == "bishop"
        n_pieces = div(n^2, 2)
    else
        n_pieces = n^2
    end
    for i in 1:n_pieces
        name = "position_" * string(i)
        domain_size = n^2
        add_variable!(csp, name, domain_size)
    end
    for i in 1:n_pieces, j in 1:n_pieces
        if (i != j)
            compatibility_tour_piece(a, b, c, d, e) = compatibility_tour(a, b, c, d, e, n, piece)
            add_constraint!(csp, i, j, compatibility_tour_piece)
        end
    end
    return csp
end

function visualize_tour(instantiation_dict::Dict{String, Int64}, n::Int64)
    for x in 1:n
        println()
        for y in 1:n
            rank::Int64 = 0
            for var in 1:length(instantiation_dict)
                if instantiation_dict["position_" * string(var)] == square_from_coord(x, y, n)
                    rank = var
                    break
                end
            end
            print(string(rank) * " "^(3 - length(string(rank))))
        end
    end
end 

visualize_tour (generic function with 1 method)

In [22]:
tour = define_tour(6, "king")
instantiation_dict = solve(tour)
visualize_tour(instantiation_dict, 6)


Status: solved

Nodes explored: 36

Solution: 
position_1: 36
position_2: 30
position_3: 35
position_4: 34
position_5: 33
position_6: 32
position_7: 31
position_8: 25
position_9: 26
position_10: 27
position_11: 28
position_12: 29
position_13: 24
position_14: 18
position_15: 23
position_16: 22
position_17: 21
position_18: 20
position_19: 19
position_20: 13
position_21: 14
position_22: 15
position_23: 16
position_24: 17
position_25: 12
position_26: 11
position_27: 6
position_28: 5
position_29: 4
position_30: 3
position_31: 2
position_32: 1
position_33: 7
position_34: 8
position_35: 9
position_36: 10


32 31 30 29 28 27 
33 34 35 36 26 25 
20 21 22 23 24 14 
19 18 17 16 15 13 
8  9  10 11 12 2  
7  6  5  4  3  1  

### 2.3) Graph coloring

In [23]:
function generate_combinations(n::Int64, k::Int64)
    if k==1
        combs = Set([[i] for i in 1:n])
    else
        combs::Set{Array{Int64, 1}} = Set()
        for c in generate_combinations(n, k-1)
            for i in 1:n
                if !(i in c)
                    new_c = copy(c)
                    push!(new_c, i)
                    push!(combs, sort(new_c))
                end
            end
        end
        return combs
    end
end

function check_clique(adj, c)
    for u in c, v in c
        if (u != v) && adj[u, v] == 0
            return false
        end
    end
    return true
end

function find_kclique(n_vertices, edges, k)
    adj = zeros(Bool, n_vertices, n_vertices)
    for (u, v) in edges
        adj[u, v] = true
        adj[v, u] = 1
    end
    for c in generate_combinations(n_vertices, k)
       if check_clique(adj, c)
            return c
        end
    end
    return 0
end

import LightGraphs

function find_maxclique(n_vertices, edges)
    g = LightGraphs.SimpleGraph(n_vertices)
    for (u, v) in edges
        LightGraphs.add_edge!(g, u, v)
    end
    k = 0
    maxclique = []
    for c in LightGraphs.maximal_cliques(g)
        if length(c) >= k
            k = length(c)
            maxclique = c
        end
    end
    return maxclique
end

┌ Info: Recompiling stale cache file /Users/guillaume/.julia/compiled/v1.0/LightGraphs/Xm08G.ji for LightGraphs [093fc24a-ae57-5d10-9952-331d41423f4d]
└ @ Base loading.jl:1190


find_maxclique (generic function with 1 method)

In [24]:
function read_graph(path::String)
    n_vertices = 0
    edges = Array{Tuple{Int64, Int64}, 1}()
    open(path) do file
        for line in eachline(file)
            split_line = split(line)
            if line[1] == 'p'
                n_vertices = parse(Int64, split_line[3])
            elseif line[1] == 'e'
                u = parse(Int64, split_line[2])
                v = parse(Int64, split_line[3])
                push!(edges, (u, v))
            end
        end
    end
    return n_vertices, edges
end

read_graph (generic function with 1 method)

In [25]:
function compatibility_coloring(csp::CSP, var1::Int64, var2::Int64, value1::Int64, value2::Int64)
    return value1 != value2
end

function define_coloring(n_vertices::Int64, edges::Array{Tuple{Int64, Int64}, 1}, n_colors::Int64)
    csp = CSP()
    for u in 1:n_vertices
        name = "color_" * string(u)
        domain_size = n_colors
        add_variable!(csp, name, domain_size)
    end
    for (u, v) in edges
        add_constraint!(csp, u, v, compatibility_coloring)
        add_constraint!(csp, v, u, compatibility_coloring)
    end
    # Symmetry constraints
    maxclique = find_maxclique(n_vertices, edges)
    if length(maxclique) > n_colors
        maxclique = maxclique[1:n_colors]
    end
    print("Max clique size ", length(maxclique))
    for (i, u) in enumerate(maxclique)
        add_constraint!(csp, u, [i])
    end
    
    sort_values(csp)
    return csp
end

define_coloring (generic function with 1 method)

In [26]:
n_vertices, edges = read_graph("graphs/jean.col")
n_colors = 9
coloring = define_coloring(n_vertices, edges, n_colors);

Max clique size 9

In [27]:
@time backtrack!(coloring, "FC")

  0.004911 seconds (2.21 k allocations: 123.631 KiB)


(false, [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1  …  -1, -1, -1, -1, -1, -1, -1, -1, -1, -1], 9)